<a href="https://colab.research.google.com/github/AlexanderMaslikhin/parsing/blob/homework2/Parsing_homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

минимум один сайт максимум все

In [95]:
from pprint import pprint
from lxml import html
import requests

# Функция принимает параметры в виде словаря с полями для url
# словарем для парсинга и базового пути к блоку новостей, так как они у всех
# разные. Возвращает словарь с названием новости и параметрами, 
# если они есть(дата, источник, ссылка).
# Сайт dzen.ru победить пока не удалось, он распознает парсинг и выкатывает страницу с 
# капчей. У сайта mail.ru новости в заголовке формируются динамически, скорее 
# всего с помощью JavaScript, поэтому спарсил news.mail.ru

def parse_news(params):
  parse_dict = {key: val for key, val in params["parse_dict"].items() if val is not None}

  reply = requests.get(params["url"], headers=params["headers"])
  dom = html.fromstring(reply.text)
  news = dom.xpath(params["base_path"])

  result = {}
  for element in news:
    new_dict = {}
    for key, req in parse_dict.items():
      new_dict[key] = str(element.xpath(req)[0])
    if new_dict['news_link'].startswith('/'):
      new_dict['news_link'] = params["url"] + new_dict['news_link']
    result[new_dict['news_name']] = new_dict
    del result[new_dict["news_name"]]["news_name"]
  return result   

keys = ("news_source", "news_name", "news_link", "news_date")

lenta_params = {
    "url": "https://lenta.ru",
    "base_path": "//div[@class='topnews']/div/a",
    "headers": {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
    },
    "parse_dict": {
        "news_source": None,
        "news_name": "./div/span[@class='card-mini__title']/text()",
        "news_link": "./@href",
        "news_date": "./div/div/time/text()"
    }   
}

mail_params = {
    "url": "https://news.mail.ru",
    "base_path": '//ul[@class="list list_type_square list_half js-module"]/li',
    "headers": {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
    },
    "parse_dict": {
        "news_source": None,
        "news_name": './span/a/text()',
        "news_link": './span/a/@href',
        "news_date": None
    }       
}

res = parse_news(lenta_params)
pprint(res)
res = parse_news(mail_params)
pprint(res)


{'В «Шиннике» рассказали о состоянии команды после драки с футболистами с Украины': {'news_date': '12:04',
                                                                                     'news_link': 'https://lenta.ru/news/2023/02/17/draka_football/'},
 'В Армении оценили возможность проведения встречи Алиева и Пашиняна в Мюнхене': {'news_date': '12:14',
                                                                                  'news_link': 'https://lenta.ru/news/2023/02/17/meeting/'},
 'В Госдуму внесли проект закона о «детях СВО»': {'news_date': '12:04',
                                                  'news_link': 'https://lenta.ru/news/2023/02/17/detisvo/'},
 'Журналист Херш рассказал о последствиях от подрыва «Северных потоков» для США': {'news_date': '12:12',
                                                                                   'news_link': 'https://lenta.ru/news/2023/02/17/sevpotokkk/'},
 'Захарова обвинила США и НАТО в трусости и цинизме из-за украинск